In [167]:
import tensorflow as tf
from keras.optimizers import SGD
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torch.nn.functional as F
import numpy as np

In [161]:
#THIS IS JUST TO GET LIST OF CHUNKED ARRAYS


def get_proportion_of_background_pixels_in_chunk(chunk):
    chunk = np.array(chunk)

    pixels = chunk.flatten()

    # How many pixels in the chunk have an intensity of 0?
    # (We will count them as background pixels)
    background_pixel_count = (pixels==0).sum()

    return background_pixel_count / len(pixels)

def split_hsi_into_chunks(hyperspectral_image, chunk_size):

    chunks = []
    # Get the width and height for the image    
    image_size = np.array(list(hyperspectral_image.shape[:-1]))
    print(image_size)

    # Get how many chunks are needed for the image
    # as an array of integers
    image_chunks = np.ceil(image_size / chunk_size).astype(np.int64)

    for x in range(image_chunks[0]):
        for y in range(image_chunks[1]):

            # Get the pixels which should be covered by the chunk.
            chunk_start = np.array([x,y]) * chunk_size
            chunk_start = np.minimum(chunk_start, image_size)
            chunk_end = np.minimum(chunk_start + chunk_size, image_size)

            image_chunk = hyperspectral_image[chunk_start[0]:chunk_end[0], chunk_start[1]:chunk_end[1], :]

            chunks.append(image_chunk)
    
    return chunks

# Remove all chunks in a list of chunks which have higher than
# background_percentage percentage of background pixels.
def remove_background_chunks_by_percentage(chunks, background_percentage):

    threshold = (1 - background_percentage)
    filtered_chunks = filter((lambda x : get_proportion_of_background_pixels_in_chunk(x) <= threshold), chunks)
    return np.array(list(filtered_chunks))


data_path_1 = np.load('C:\\Users\\bendo\\Desktop\\uni\\ProjectA\\sugarcane\\TEST\\con_pro\\Q78_1_c_irradiance.npy')
data_path_2 = np.load('C:\\Users\\bendo\\Desktop\\uni\\ProjectA\\sugarcane\\TEST\\dis_pro\\Q78_1_irradiance.npy') 

chunks_c = split_hsi_into_chunks(data_path_1, 32)
chunks_d = split_hsi_into_chunks(data_path_2, 32)

# Remove all chunks with more than 90% background content
chunks_c = remove_background_chunks_by_percentage(chunks_c, 0.1)
chunks_d = remove_background_chunks_by_percentage(chunks_d, 0.1)

[1088 2048]
[1088 2048]


In [162]:
joint_chunks = np.concatenate((chunks_c,chunks_d))

c_l = []
d_l = []

#need to figure this out, just away to get list of classes for now

classes = ('contorl', 'disease')

for x in range(len(chunks_c)):
    c_l.append([0])

for x in range(len(chunks_d)):
    d_l.append([1])

classes_lists = c_l + d_l

print('class list: ',len(classes_lists), 'chunks_list', len(joint_chunks)) #should be same


class list:  537 chunks_list 537


In [163]:
#create x and y for data

x = torch.from_numpy(joint_chunks)
y = torch.FloatTensor(classes_lists)

In [164]:
#reshape x for pytorch
#WARNING THIS COULD BE WRONG

x = torch.reshape(x,(len(joint_chunks),161,32,32))

In [165]:
#training and test 80/20 split
#also shuffle


X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible

len(X_train), len(X_test), len(y_train), len(y_test)

(429, 108, 429, 108)

In [166]:
#load into dataloader for pytroch

#code from >> https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

my_dataset = TensorDataset(X_train,y_train)
my_dataloader = DataLoader(my_dataset) # create your dataloader


In [168]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(161, 32, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()


In [131]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [133]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(my_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


In [134]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [141]:
my_dataset = TensorDataset(X_test,y_test)
my_dataset_test = DataLoader(my_dataset)
dataiter = iter(my_dataset_test)
images, labels = next(dataiter)

net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [142]:
outputs = net(images)

In [143]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [145]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in my_dataset_test:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 67 %
